In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from tass import InterTASSReader
from analysis import print_maxent_features, print_feature_weights_for_item
import matplotlib.pyplot as plt


def load_data():
    tassDev = InterTASSReader("InterTASS/ES/intertass-ES-development-tagged.xml")
    X_dev = list(tassDev.X())
    y_dev = list(tassDev.y())

    tassTrain = InterTASSReader("InterTASS/ES/intertass-ES-train-tagged.xml")
    X_train = list(tassTrain.X())
    y_train = list(tassTrain.y())

    return X_dev, y_dev, X_train, y_train

In [3]:
X_dev, y_dev, X_train, y_train = load_data()

In [10]:
from classifier import SentimentClassifier

sentimentClassifier = SentimentClassifier(clf = 'maxent')
sentimentClassifier.fit(X_train, y_train, X_dev, y_dev)
pipeline = sentimentClassifier._pipeline

/home/pedro/.virtualenvs/pln/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pedro/.virtualenvs/pln/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pedro/.virtualenvs/pln/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/pedro/.virtualenvs/pln/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
vect = pipeline.named_steps['vect']
clf = pipeline.named_steps['clf']

In [15]:
features = vect.get_feature_names()
coef = clf.coef_

# Ejercicio 4

In [28]:
print_maxent_features(vect, clf, 10)

N:
	! gracias mejor buena primer ([-0.83044966 -0.45699408 -0.44051692 -0.36302183 -0.35821204 -0.35254213
 -0.33985407 -0.32972154 -0.31863342 -0.29588705])
	NOT_ni odio mal triste no ([0.27003069 0.30371244 0.30411346 0.31481355 0.32240395 0.33303664
 0.43855286 0.53304974 0.58158978 0.84728895])
NEU:
	! gracias hoy hacer quiero ([-0.44498438 -0.43189308 -0.3006553  -0.25977253 -0.24825548 -0.22946173
 -0.22896164 -0.21985821 -0.20754978 -0.20266067])
	hombre NOT_pasa vez sido aunque ([0.1743627  0.18099543 0.20616629 0.21507509 0.21525027 0.22326693
 0.23003609 0.24993635 0.29292053 0.3388004 ])
NONE:
	no ... ! mal , ([-0.62669427 -0.42365885 -0.35421905 -0.27651121 -0.26400719 -0.25237283
 -0.2320382  -0.22947012 -0.22896592 -0.22804821])
	jugar alguna semana " ? ([0.19184573 0.19904559 0.23808361 0.24237828 0.28254938 0.30848274
 0.313934   0.31416905 0.40413254 0.89216705])
P:
	no ? triste mal ni ([-0.68846476 -0.56128492 -0.37451977 -0.30823815 -0.24716562 -0.23501746
 -0.229550

# Analisis

En general tienen sentido, la mayoría de los features son palabras con clara carga semantica. *mal*, *triste*, *no*, se asocian con sentimientos negativos que es lo que uno esperaría. Lo mismo pasa con las palabras asociadas a sentimientos positivos. Sin embargo las palabras asociadas con **NONE** y **NEU** no son tan claras, por lo menos las que tienen peso positivo.

# Ejercicio 5

In [60]:
import pandas as pd

y_pred = pipeline.predict(X_dev)

y_prob = pipeline.predict_proba(X_dev)

probs = {"N":0,"NEU":1,"NONE":2,"P":3}

errors = []
for i, (x, y1, y2, y3) in enumerate(zip(X_dev, y_dev, y_pred, y_prob)):
    if y1 != y2:
        errors.append({
            'index': i,
            'item': x,
            'true': y1,
            'pred': y2,
            'pred_prob': y3[probs[y2]],
            'true_prob': y3[probs[y1]]})

errdf = pd.DataFrame(errors)
errdf['len'] = errdf['item'].apply(lambda x: len(x))


In [67]:
display(errdf.sort_values('pred_prob', ascending=False)[:20])

,index,item,pred,pred_prob,true,true_prob,len
63,146,La persona vale para algo. Otra cosa es que no...,N,0.668041,P,0.147123,134
82,191,@Nadieelosabe Vale vuelvo a preguntar. No sabi...,N,0.649930,NONE,0.081780,67
93,217,@LovNaty Tu vida ha parido a un grandisimo hij...,P,0.643372,N,0.197041,91
154,333,No silencio lo que calla. Es la vida suspendid...,N,0.641812,NONE,0.066547,126
190,437,Esta decidido. ¡Habrá Modo Carrera en el canal...,P,0.637488,NONE,0.140280,133
145,320,A mí nunca me podrán hacer una broma porque no...,N,0.636474,NONE,0.106083,88
41,100,"@gilthoniel_1987 Si estoy trabajando 😭, con vi...",P,0.634568,NEU,0.091338,73
119,270,@MV3ga hay cosas del hilo con las que discrepo...,N,0.629306,NONE,0.078026,133
189,430,@TichKhan_ d los cines donde la hacen este fin...,N,0.627497,P,0.147824,138
185,417,"@CucoRguez Ya, estoy jugando al ratón y al gat...",N,0.624486,P,0.158688,120


## Analisis de un error

In [92]:
x = X_dev[146]
tweet = x

* 2/3 de la palabra no son features

In [100]:
len(tweet.split()) #24
len(set(tweet.split()) - set(features)) #16

'la persona vale para algo. otra cosa es que no te creas lo maravilloso y ese arte que puedes desplegar..fingelo hasta que se exprese.\n'

* Palabras mal takenizadas y desconocidas

In [93]:
from nltk.corpus import stopwords
from tokenizer import CustomTokenizer

tokenizer = CustomTokenizer()
words = set(tokenizer.tokenize(tweet)) - set(features)
words


{'NOT_arte',
 'NOT_desplegar',
 'NOT_exprese',
 'NOT_fingelo',
 'NOT_maravilloso',
 'NOT_puedes'}

## Features involucrados y sus pesos

In [86]:
print_feature_weights_for_item(vect,clf,x)

. [ 0.1357706  -0.1417081  -0.22804821 -0.08348188]
NOT_.. [ 0.00345008 -0.02806548  0.08089263 -0.05376528]
NOT_creas [-0.07001599 -0.01495925 -0.00889087  0.09251963]
cosa [ 0.30371244 -0.14239191 -0.1394941  -0.07827462]
no [ 0.84728895 -0.17918136 -0.62669427 -0.68846476]
persona [ 0.01349378  0.03250738 -0.15414068  0.05096104]
vale [ 0.05107103 -0.0761001   0.10967915 -0.11173736]


* La presencia del token *no*, incrementa el demaciado el peso de **N**.

In [115]:
new_x = x.replace('no','')
pipeline.predict_proba([x, new_x])


array([[0.66804128, 0.10540516, 0.07943104, 0.14712252],
       [0.42891679, 0.11282748, 0.12079086, 0.33746486]])

Como es de esperar si removemos ese token la probabilidad de **N** disminuye. Sinembargo no lo suficiente como para que se determine correctamente la clase.

In [117]:
new_x = x.replace('desplegar..fingelo','desplegar .. fingelo')
pipeline.predict_proba([x, new_x])

array([[0.66804128, 0.10540516, 0.07943104, 0.14712252],
       [0.66804128, 0.10540516, 0.07943104, 0.14712252]])

Vemos que corregir la tokenizacion no tiene ningun efecto. Quizas con data augmentation se podría obtener un mejor resultado, ya que hay varias palabras desconocidas.